In [1]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import math

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import spatial

from sentence_transformers import SentenceTransformer
from sentence_transformers import  util

import scipy.spatial

import pandas as pd
import numpy as np


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdulwahabmac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/abdulwahabmac/Desktop/MyFiles/Projects/Training/Tuwaiq/Tuwaiq-LLM-28-July/.env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [52]:

def preprocess_query(query):
    # Tokenization, stop word removal, and stemming
    stop_words = set(stopwords.words('arabic'))
    stemmer = PorterStemmer()
    tokens = query.lower().split()
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)


In [55]:

def hybrid_search(query, top_k=3, weight_st=0.5, weight_tfidf=0.5):
    
    # Encode the query using SentenceTransformers
    query_preprocessed = preprocess_query(query)
    query_embedding = model.encode([query_preprocessed], convert_to_tensor=True)

    # Compute cosine similarities between the query and the documents using SentenceTransformers
    cosine_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

    # Transform the query using the TF-IDF vectorizer
    query_tfidf = tfidf_vectorizer.transform([query_preprocessed])

    # Calculate cosine similarity between the query and the documents using TF-IDF
    tfidf_similarities = cosine_similarity(query_tfidf, tfidf_matrix)[0]
    #print(tfidf_similarities)

    # Combine the scores with weights
    combined_scores = (weight_st * cosine_scores.cpu().numpy() + weight_tfidf * tfidf_similarities) / (weight_st + weight_tfidf)
    #print(combined_scores)

    # Normalize scores (optional)
    combined_scores = (combined_scores - np.min(combined_scores)) / (np.max(combined_scores) - np.min(combined_scores))

    # Find the top_k highest scores
    top_results = np.argpartition(-combined_scores, range(top_k))[:top_k]
    #print(top_results)

    print(f"Query: {query}\nTop {top_k} most similar arabic_documents:")
    for idx in top_results:
        print("*"*30) 
        print()
        print()
        print(f"Document: {arabic_documents[idx]},")
        print() 
        print(f"Combined Score: {combined_scores[idx]:.4f}")
        print()
        print()
        print() 
        print("*"*30) 

    return top_results, combined_scores



In [27]:
df = pd.read_csv('ar_reviews_100k.tsv', sep='\t')

In [28]:
# from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

In [29]:
arabic_documents  = [x for x in df.text][:10000]

In [30]:

# Encode the documents using SentenceTransformers
document_embeddings = model.encode(arabic_documents, convert_to_tensor=True)


In [31]:

# Create a TF-IDF Vectorizer and fit on the documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(arabic_documents)


In [56]:
for _ in df.text[100:200]:
    print()
    print()
    print()
    print(_)
    print()
    print()
    print()




رائع جدا . استمعت جدا كل شي جميل. الانتظار عند الاصنصيرات






جيد جدا. تصميم ال يلا وموقعها. عدم وجود سجاد على الأرض






فلة إحجاج . الموقع وطاقم العمل. نظافة الموكيت






كتاب جميل وخفيف .. هو تجربتي الثانية مع عبدالوهاب مطاوع. استمتعت به للحد الأاقصى






شئ مرعب انك تشوف نهايتك. تعيشها كل يوم. وتعيش آلامها. شئ مرعب انك تكون عارف حبايبك هيموتوا ازاي. في اللحظات دي الجهل مفيد. اكتر من المعرفه ..... هل تعرف محمود السمنودي؟. فلتحمد الله انك لا تعرفه.






من اجمل الروايات اللى قرتها فى حياتى ومش لاقية حاجة بمستواها لحد دلوقتى






الأحداث هنا اكتر بكتير من الجزء الأول والأسلوب احلى كمان ،،عجبنى خياله جدا فى عالم الجن بصورة مختلفة تماما عن المعروف مستنيين العائد وشكلها هتتأخر وهننسى الأحداث :






اكثر من راااااااااائع من حيث الاسلوب والسرد ومفيييييييد جداااااااااااااااااااااااا






جيد جدا . موقع الفندق والخدمات بشكل عام. سعر الفنادق جميعها بشكل عام جدا مرتفع






جيد جدا. . موظف في المطعم لا يحضرني اسمه كان تعامله سيء جدا جدا،






رائع رائع رائع .. غريب أن شخصية مثل عب

In [51]:
# df.query("")
# Document: رائع حد الهذيان,


In [57]:

# Example usage with evaluation:
while True:
    user_query = input("Enter your query (or type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break
    hybrid_search(user_query,top_k=10)

    predicted_ranks, _ = hybrid_search(user_query)


Enter your query (or type 'exit' to quit):  الديكور مره يجننن


Query: الديكور مره يجننن
Top 10 most similar arabic_documents:
******************************


Document: مريح جدا . الديكور جميل جدا.,

Combined Score: 1.0000



******************************
******************************


Document: جيد جدا. الديكور عصري وجميلالفندق هادىء.,

Combined Score: 0.6203



******************************
******************************


Document: المبرومة حقتهم مره خطيييييرة .. التشيز كيك مره حلو وكيكة الفواكة مره حلووة دائما ناخذ منها,

Combined Score: 0.5344



******************************
******************************


Document: جميييل . الديكور ألوانه رايقه إطلالة الفندق خياليه. شيئ,

Combined Score: 0.5240



******************************
******************************


Document: قرأتي ماريا اول مره في و من ساعتها قرأتها اكثر من مرات و كل مره بتأثر فيا اكتر,

Combined Score: 0.4836



******************************
******************************


Document: جيد. الديكور ملفت الهدوء و النظافة . الرسبشن يجب تدريبهم اكثر لتعاملهم مع الزوار,

Combi

KeyboardInterrupt: Interrupted by user